In [19]:
import numpy as np
import sys

## With DualSimplexSolver

In [1]:
from functions.DualSimplexSolver import DualSimplexSolver
if __name__ == "__main__":
    try:
        example_obj_type = 'max'
        example_c = [-1, 2]
        example_A = [
            [5, 4],
            [1, 5],
        ]
        example_relations = ['>=', '=']
        example_b = [20,10]

        solver = DualSimplexSolver(example_obj_type, example_c, example_A, example_relations, example_b)
        solver.solve()
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        import traceback
        traceback.print_exc()


The standard Dual Simplex method might not apply directly or may require Phase I.
--- Starting Dual Simplex Method ---

--- Iteration 0 ---
      BV        Z       x1       x2       s1       s2       s3      RHS
------------------------------------------------------------------------
       Z    1.000    1.000   -2.000    0.000    0.000    0.000    0.000
      s1    0.000   -5.000   -4.000    1.000    0.000    0.000  -20.000
      s2    0.000    1.000    5.000    0.000    1.000    0.000   10.000
      s3    0.000   -1.000   -5.000    0.000    0.000    1.000  -10.000
------------------------------------------------------------------------

Step: Select Pivot Row (Leaving Variable)
   RHS values (b): [-20.  10. -10.]
   Most negative RHS is -20.000 in Row 1 (Basic Var: s1).
   Leaving Variable: s1 (Row 1)

Step: Select Pivot Column (Entering Variable) using Ratio Test
   Pivot Row (Row 1) coefficients (excluding Z, RHS): [-5. -4.  1.  0.  0.]
   Objective Row coefficients (excluding Z, 

## Without DualSimplexSolver

In [2]:
from functions.get_user_input import get_user_input
from functions.solve_lp_via_dual import solve_lp_via_dual
from functions.solve_primal_directly import solve_primal_directly


use_example = True # Set to False to use user input

if use_example:
     print("--- Using Example Problem ---")
     example_obj_type = 'max'
     example_c = [-1, 2]
     example_A = [
          [5, 4],
          [1, 5],
     ]
     example_relations = ['>=', '=']
     example_b = [20,10]

     print(f"Objective: {example_obj_type} Z = {' + '.join([f'{co:.2f}*x{i+1}' for i, co in enumerate(example_c)])}")
     print("Constraints:")
     for i in range(len(example_A)):
          print(f"  {' + '.join([f'{aij:.2f}*x{j+1}' for j, aij in enumerate(example_A[i])])} {example_relations[i]} {example_b[i]:.2f}")
     print("------------------------\n")

     obj_type, c, A, relations, b = example_obj_type, example_c, example_A, example_relations, example_b
else:
     try:
          obj_type, c, A, relations, b = get_user_input()
          print("\n--- Problem Received ---")
          print(f"Objective: {obj_type} Z = {' + '.join([f'{co:.2f}*x{i+1}' for i, co in enumerate(c)])}")
          print("Constraints:")
          for i in range(len(A)):
               print(f"  {' + '.join([f'{aij:.2f}*x{j+1}' for j, aij in enumerate(A[i])])} {relations[i]} {b[i]:.2f}")
          print("------------------------\n")
     except Exception as e:
          print(f"\nAn error occurred during input: {e}")
          exit()

# Solve using the dual method
status, message, primal_sol, dual_sol, obj_val = solve_lp_via_dual(obj_type, c, A, relations, b)

print("\n" + "="*40)
print("          FINAL RESULT")
print("="*40)
print(f"Status: {status} ({message})")
if status == 0:
     print(f"Optimal Primal Objective Value: {obj_val:.6f}")
     print("Primal Variables (x*):")
     if primal_sol:
          for var, val in primal_sol.items():
               print(f"   {var}: {val:.6f}")
     print("Dual Variables (p*):")
     if dual_sol:
          # Map dual vars back to original constraints if possible (tricky due to splitting '=')
          # For simplicity, just print the p's as solved.
          for var, val in dual_sol.items():
               print(f"   {var}: {val:.6f}") # p_i corresponds to the i-th *standardized* constraint
print("="*40)

# Optional: Solve directly for comparison
print("\n--- Solving Primal Directly (for comparison) ---")
status_direct, msg_direct, psol_direct, _, obj_direct = solve_primal_directly(obj_type, c, A, relations, b)
if status_direct == 0:
     print(f"Direct Solve Optimal Value: {obj_direct:.6f}")
     print("Direct Solve Primal Variables:")
     for var, val in psol_direct.items():
          print(f"   {var}: {val:.6f}")
else:
     print(f"Direct Solve Failed: {msg_direct}")
print("="*40)

--- Using Example Problem ---
Objective: max Z = -1.00*x1 + 2.00*x2
Constraints:
  5.00*x1 + 4.00*x2 >= 20.00
  1.00*x1 + 5.00*x2 = 10.00
------------------------

--- Step 1: Formulate the Dual Problem ---
   Primal is Max. Converting to Min by negating objective coefficients.
   Adjusting primal constraints to >= form for dual formulation:
      Constraint 1: Keeping as >= ( [5. 4.] >= 20.0 )
      Constraint 2: Splitting '=' into >= and <= 
         Part 1: [1. 5.] >= 10.0
         Part 2: [-1. -5.] >= -10.0 (from [1. 5.] <= 10.0)

   Dual Problem Formulation:
      Objective: Maximize p * [20.00, 10.00, -10.00]
      Subject to:
         5.00*p1 + 1.00*p2 + -1.00*p3 <= 1.00
         4.00*p1 + 5.00*p2 + -5.00*p3 <= -2.00
      p_i >= 0 for i=1..3

--- Step 2: Solve the Dual Problem using SciPy linprog ---

   Optimal Dual Solution Found:
      Dual Variables (p*):
         p1 = 0.333333
         p2 = 0.000000
         p3 = 0.666667
      Optimal Dual Objective Value (Max p*b): 0.000